# 数据查询的四大类方法

从dateframe中查询，如果只查询一行、一列返回的是pd.Serires；如果查询多行、多列，返回的是pd.DataFrame

In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.DataFrame({"id" : [1001,1002,1003,1004,1005,1006],  
  "date" : pd.date_range('20130102', periods=6), 
  "city" : ['Beijing ', 'SH', ' guangzhou ', 'Shenzhen', 'shanghai', 'BEIJING '], 
  "age" : [23,44,54,32,34,32], 
  "category" : ['100-A','100-B','110-A','110-C','210-A','130-F'], 
  "price" : [1200,np.nan,2133,5433,np.nan,4432]}, 
  columns =['id','date','city','category','age','price'])
df.head()

,id,date,city,category,age,price
0,1001,2013-01-02,Beijing,100-A,23,1200.0
1,1002,2013-01-03,SH,100-B,44,NaN
2,1003,2013-01-04,guangzhou,110-A,54,2133.0
3,1004,2013-01-05,Shenzhen,110-C,32,5433.0
4,1005,2013-01-06,shanghai,210-A,34,NaN


## 1.切片查询

### 列索引
* 列索引是最常见的索引形式，一般通过 [] 来实现。通过 [列名] 可以从 DataFrame 中取出相应的列，返回值为 Series
* 如果要取出多个列，则可以通过 [列名组成的列表] ，其返回值为一个 DataFrame 
* 若要取出单列，且列名中不包含空格，则可以用 .列名 取出，这和 [列名] 是等价的

In [13]:
df['date']
df[['date','city']]
df.date

0   2013-01-02
1   2013-01-03
2   2013-01-04
3   2013-01-05
4   2013-01-06
5   2013-01-07
Name: date, dtype: datetime64[ns]

### 行索引
* dataframe不可以使用切片对行索引进行查询，查询出来的series可以
* series的行索引访问方式与列表类似，但是增加了索引标签的访问

In [28]:
s1 = pd.Series([1, 2, 3, 4, 5, 6],index=['a', 'b', 'a', 'a', 'a', 'c'])
s1['a']
s1[['b','c']] ## 取多个索引
s1['c': 'b': -2] ## 此种方式，包含端点元素，但是端点值不能存在重复，否则会报错如：s1['a':'b':-1]
s1.sort_index()['a': 'b'] ## 可以排序后使用

a    1
a    3
a    4
a    5
b    2
dtype: int64

In [43]:
s2 = pd.Series([1, 2, 3, 4, 5, 6])
s2[0]
s2[[0,3]]
s2[0:3:2]
s2[0:] 
s2[1:-1] ## 不包含右端点，左闭右开

1    2
2    3
3    4
4    5
dtype: int64

## 2.loc按照行索引标签值查询

* loc 索引器的一般形式是 loc[*, *] ，其中第一个 * 代表行的选择，第二个 * 代表列的选择，如果省略第二个位置写作 loc[*] ，这个 * 是指行的筛选。其中， * 的位置一共有五类合法对象，分别是：单个元素、元素列表、元素切片、布尔列表以及函数

In [4]:
df.set_index('id',inplace=True)

In [60]:
df

,date,city,category,age,price
id,,,,,
1001,2013-01-02,Beijing,100-A,23,1200.0
1002,2013-01-03,SH,100-B,44,NaN
1003,2013-01-04,guangzhou,110-A,54,2133.0
1004,2013-01-05,Shenzhen,110-C,32,5433.0
1005,2013-01-06,shanghai,210-A,34,NaN
1006,2013-01-07,BEIJING,130-F,32,4432.0


### 使用单个label值查询数据

In [78]:
df.loc[1003] #单个元素 , 如果index不重复返回Series
df.loc[1003,'city'] # 和列名一起

' guangzhou '

### 使用值列表批量查询


In [79]:
df.loc[[1002,1004]] #元素列表 等价于 df.loc[[1002,1004],:]

,date,city,category,age,price
id,,,,,
1002,2013-01-03,SH,100-B,44,NaN
1004,2013-01-05,Shenzhen,110-C,32,5433.0


### 使用数值区间进行范围查询


In [80]:
df.loc[1002:1004] #元素切片 ,如果是唯一值的起点和终点字符，那么就可以使用切片，并且包含两个端点，如果不唯一则报错

,date,city,category,age,price
id,,,,,
1002,2013-01-03,SH,100-B,44,NaN
1003,2013-01-04,guangzhou,110-A,54,2133.0
1004,2013-01-05,Shenzhen,110-C,32,5433.0


### 使用条件表达式查询

In [81]:
df.loc[df.age>40] #根据条件返回的布尔值查询
df.loc[df.city.isin(['SH','Shenzhen'])] ##isin必须是str类型

,date,city,category,age,price
id,,,,,
1002,2013-01-03,SH,100-B,44,NaN
1004,2013-01-05,Shenzhen,110-C,32,5433.0


### 调用函数查询

In [82]:
def condition(x):
    condition1 = x.age>40
    condition2 = x.city=='Shenzhen'
    con = condition2 & (~condition1)
    return con
df.loc[condition] # 使用函数

,date,city,category,age,price
id,,,,,
1004,2013-01-05,Shenzhen,110-C,32,5433.0


In [83]:
df.loc[lambda x:1004,lambda x:'city']

'Shenzhen'

## 3.iloc按照行位置查询

iloc 的使用与 loc 完全类似，只不过是针对位置进行筛选，在相应的 * 位置处一共也有五类合法对象，分别是：整数、整数列表、整数切片、布尔列表以及函数

In [85]:
df.iloc[1, 1] 

'SH'

In [90]:
df.iloc[[0, 1], [0, 1]]
df.iloc[0:2, 1:3] ## 左闭右开

,city,category
id,,
1001,Beijing,100-A
1002,SH,100-B


In [91]:
df.iloc[lambda x: slice(1, 4)]

,date,city,category,age,price
id,,,,,
1002,2013-01-03,SH,100-B,44,NaN
1003,2013-01-04,guangzhou,110-A,54,2133.0
1004,2013-01-05,Shenzhen,110-C,32,5433.0


在使用布尔列表的时候要特别注意，不能传入 Series 而必须传入序列的 values ，否则会报错。因此，在使用布尔筛选的时候还是应当优先考虑 loc 的方式

In [94]:
df.iloc[(df.age>40).values]

,date,city,category,age,price
id,,,,,
1002,2013-01-03,SH,100-B,44,NaN
1003,2013-01-04,guangzhou,110-A,54,2133.0


## 4.query

* 使用query方法，将字符串形式的查询表达式传入进行查询，表达式的执行结果需返回布尔值，该种方法无需重复使用DataFrame的名字来引用列名
* 在 query 表达式中，帮用户注册了所有来自 DataFrame 的列名，所有属于该 Series 的方法都可以被调用，和正常的函数调用并没有区别
* 必须使用单引号扩起查询表达式
* 对于含有空格的列名，需要使用 \`col name\` 的方式进行引用
* 可以使用or, and, in, not in等
* 对于 query 中的字符串，如果要引用外部变量，只需在变量名前加 @ 符号

In [5]:
df

,date,city,category,age,price
id,,,,,
1001,2013-01-02,Beijing,100-A,23,1200.0
1002,2013-01-03,SH,100-B,44,NaN
1003,2013-01-04,guangzhou,110-A,54,2133.0
1004,2013-01-05,Shenzhen,110-C,32,5433.0
1005,2013-01-06,shanghai,210-A,34,NaN
1006,2013-01-07,BEIJING,130-F,32,4432.0


In [8]:
df.query('age>40')
df.query('age>age.mean()')

,date,city,category,age,price
id,,,,,
1002,2013-01-03,SH,100-B,44,NaN
1003,2013-01-04,guangzhou,110-A,54,2133.0


In [28]:
df.query('(age>30) and (price>3000) or (city in ["SH","shanghai"])')

,date,city,category,age,price
id,,,,,
1002,2013-01-03,SH,100-B,44,NaN
1004,2013-01-05,Shenzhen,110-C,32,5433.0
1005,2013-01-06,shanghai,210-A,34,NaN
1006,2013-01-07,BEIJING,130-F,32,4432.0


In [20]:
low = 30
df.query(('age>@low' and 'price>3000') or '(city==["SH"])')

,date,city,category,age,price
id,,,,,
1004,2013-01-05,Shenzhen,110-C,32,5433.0
1006,2013-01-07,BEIJING,130-F,32,4432.0


In [27]:
df.query('index==[1002,1003]')
df.query('index in [1002,1003]')

,date,city,category,age,price
id,,,,,
1002,2013-01-03,SH,100-B,44,NaN
1003,2013-01-04,guangzhou,110-A,54,2133.0
